# EXP1: BASELINE

- Missing : num -> median , cat -> 'unknown' , outlier -> clip by percentiles (1 - 99%)
- Transform/scale : log1p for Skewness features. 
- Encoding : orinal encoding for categorical 
- NO FE , PCA , SMOTE 
- Model : Logistic, Linear/Kernel SVM, Decision Tree, Random Forest, XGBoost, LightGBM, CatBoost, MLP. 
- CV: 5-fold 
- metric: ROC-AUC, accuracy , f1-score , recall , precision 

# EXP2: Feature Engineering 

- Add feature :  .... 
- Keep all step of EXP1 
- Model, metrics , CV like EXP1

# EXP 3a : PCA on Numerical 


# EXP 3b : IV Values  - reduce demisions

# EXP 4a: Feature Engineering and PCA 

# EXP 4b : FE and IV values 

# EXP 4c : FE , PCA , IV values 

# EXP 5 : Imbalance 

# EXP 6 : combine ... 

# EXP 7 : HYPER TUNNING PARAMETERS
